In [17]:
import tensorflow as tf

filename_queue=tf.train.string_input_producer(
    ['data-04-zoo.csv'],shuffle=False,name='filename_queue')

reader=tf.TextLineReader()
key,value=reader.read(filename_queue)

record_defaults=[[0.] for _ in range(17)]
xy=tf.decode_csv(value,record_defaults=record_defaults)

train_x_data,train_y_data= tf.train.batch([xy[0:-1],xy[-1:]],batch_size=10)

In [21]:
nb_classes=7

X=tf.placeholder(tf.float32,[None,16])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W=tf.Variable(tf.random_normal([16,nb_classes]),name='weight')
b=tf.Variable(tf.random_normal([nb_classes]),name='bias')

logits=tf.matmul(X,W)+b
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [37]:
correct_prediction=(tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y_one_hot,1)))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

sess=tf.Session()
sess.run(tf.global_variables_initializer())

coord=tf.train.Coordinator()
threads=tf.train.start_queue_runners(sess=sess,coord=coord)

for step in range(200):
    x_data,y_data=sess.run([train_x_data,train_y_data])
    sess.run(optimizer,feed_dict={X:x_data,Y:y_data})
    if step%10==0:
        loss,acc=sess.run([cost,accuracy],feed_dict={X:x_data,Y:y_data})
        print(" Step: {:5}\t Loss: {: .3f}\t Acc: {: .2%}".format(step,loss,acc))

pred=sess.run(tf.argmax(hypothesis,1),feed_dict={X:x_data})

for p,y in zip(pred,y_data.flatten()):
    print(p==int(y),p,int(y))
        
coord.request_stop()
coord.join(threads)

 Step:     0	 Loss:  5.501	 Acc:  0.00%
 Step:    10	 Loss:  1.742	 Acc:  70.00%
 Step:    20	 Loss:  1.410	 Acc:  70.00%
 Step:    30	 Loss:  1.135	 Acc:  70.00%
 Step:    40	 Loss:  0.900	 Acc:  70.00%
 Step:    50	 Loss:  0.709	 Acc:  70.00%
 Step:    60	 Loss:  0.562	 Acc:  90.00%
 Step:    70	 Loss:  0.453	 Acc:  90.00%
 Step:    80	 Loss:  0.374	 Acc:  90.00%
 Step:    90	 Loss:  0.316	 Acc:  90.00%
 Step:   100	 Loss:  0.274	 Acc:  90.00%
 Step:   110	 Loss:  0.242	 Acc:  90.00%
 Step:   120	 Loss:  0.217	 Acc:  90.00%
 Step:   130	 Loss:  0.198	 Acc:  100.00%
 Step:   140	 Loss:  0.182	 Acc:  100.00%
 Step:   150	 Loss:  0.168	 Acc:  100.00%
 Step:   160	 Loss:  0.157	 Acc:  100.00%
 Step:   170	 Loss:  0.148	 Acc:  100.00%
 Step:   180	 Loss:  0.139	 Acc:  100.00%
 Step:   190	 Loss:  0.132	 Acc:  100.00%
True 2 2
False 6 2
True 3 3
True 0 0
True 0 0
True 1 1
True 0 0
True 5 5
True 0 0
False 2 6


In [43]:
print(sess.run(tf.argmax(Y_one_hot,1),feed_dict={Y:y_data}))
print(y_data)
print(sess.run(tf.cast(y_data.flatten(),tf.int32)))

[2 2 3 0 0 1 0 5 0 6]
[[ 2.]
 [ 2.]
 [ 3.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 5.]
 [ 0.]
 [ 6.]]
[2 2 3 0 0 1 0 5 0 6]
